In [2]:
import torch
from torch import nn
from torchvision.datasets import CIFAR100
from torchvision import transforms
from torch.utils.data import DataLoader

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")
print(f"Torch version: {torch.__version__}")

Device: cuda
Torch version: 2.8.0+cu126


# Datapipeline

In [3]:
CIFAR100_MEAN = (0.5071, 0.4867, 0.4408)
CIFAR100_STD = (0.2675, 0.2565, 0.2761)

In [4]:
train_transformations = transforms.Compose([transforms.RandomCrop(size=32, padding=4),
                                      transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.02),
                                      transforms.RandomHorizontalFlip(p=0.5),
                                      transforms.ToTensor(),
                                      transforms.Normalize(CIFAR100_MEAN, CIFAR100_STD)])

test_transformations = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(CIFAR100_MEAN, CIFAR100_STD),
])

In [5]:
train_data = CIFAR100(root="data",
                      train=True,
                      transform=train_transformations,
                      download=True)

test_data = CIFAR100(root="data",
                      train=False,
                      transform=test_transformations,
                      download=True)

100%|██████████| 169M/169M [00:04<00:00, 37.7MB/s]


In [13]:
NUM_WORKERS = 2

train_dataloader = DataLoader(dataset=train_data,
                              batch_size=32,
                              shuffle=True,
                              num_workers=NUM_WORKERS,
                              pin_memory=True,
                              persistent_workers=(NUM_WORKERS > 0),
                              drop_last=True)

test_dataloader = DataLoader(dataset=test_data,
                              batch_size=32,
                              shuffle=False,
                              num_workers=NUM_WORKERS,
                              pin_memory=True,
                              persistent_workers=(NUM_WORKERS > 0),
                              drop_last=False)

In [14]:
len(train_dataloader), len(test_dataloader)

(1562, 313)

In [15]:
image, label = next(iter(train_dataloader))
flattener = nn.Flatten()
print(image[0].shape)
flattener(image[0]).shape

torch.Size([3, 32, 32])


torch.Size([3, 1024])

# Importing helper functions


In [9]:
from pathlib import Path
import requests

if Path("helper_functions.py").is_file():
  print("helper_functions.py exists")
else:
  print("Downloading helper_functions.py")
  request = requests.get("https://raw.githubusercontent.com/adii11001/AIML-experiments/refs/heads/main/CIFAR100/helper_functions.py")
  with open("helper_functions.py", "wb") as f:
    f.write(request.content)

In [10]:
from helper_functions import *

# Creating Model

In [25]:
class TheImageClassification_inatorV0(nn.Module):
  def __init__(self):
    super().__init__()
    self.block_1 = nn.Sequential(nn.Conv2d(in_channels=3,
                                          out_channels=64,
                                          kernel_size=3,
                                          padding=1),
                                 nn.BatchNorm2d(num_features=64),
                                 nn.ReLU(),
                                 nn.Conv2d(in_channels=64,
                                          out_channels=64,
                                          kernel_size=3,
                                          padding=1),
                                 nn.BatchNorm2d(num_features=64),
                                 nn.ReLU(),
                                 nn.MaxPool2d(kernel_size=2,
                                              stride=2))
    self.block_2 = nn.Sequential(nn.Conv2d(in_channels=64,
                                          out_channels=64,
                                          kernel_size=3,
                                          padding=1),
                                 nn.BatchNorm2d(num_features=64),
                                 nn.ReLU(),
                                 nn.Conv2d(in_channels=64,
                                          out_channels=64,
                                          kernel_size=3,
                                          padding=1),
                                 nn.BatchNorm2d(num_features=64),
                                 nn.ReLU(),
                                 nn.MaxPool2d(kernel_size=2,
                                              stride=2))
    self.block_3 = nn.Sequential(nn.Conv2d(in_channels=64,
                                          out_channels=128,
                                          kernel_size=3,
                                          padding=1),
                                 nn.BatchNorm2d(num_features=128),
                                 nn.ReLU(),
                                 nn.Conv2d(in_channels=128,
                                          out_channels=128,
                                          kernel_size=3,
                                          padding=1),
                                 nn.BatchNorm2d(num_features=128),
                                 nn.ReLU(),
                                 nn.MaxPool2d(kernel_size=2,
                                              stride=2))

    self.classifier = nn.Sequential(nn.Flatten(),
                                    nn.Dropout(p=0.5),
                                    nn.Linear(in_features=128 * 4 * 4,
                                              out_features=100))
    self.layer_stack = nn.Sequential(self.block_1,
                                     self.block_2,
                                     self.block_3,
                                     self.classifier)

  def forward(self, X:torch.Tensor):
    return self.layer_stack(X)

model = TheImageClassification_inatorV0().to(device)
model

TheImageClassification_inatorV0(
  (block_1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block_2): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block_3): Sequentia

In [26]:
with torch.inference_mode():
    pred = model(image.to(device))
print(pred.shape)

torch.Size([32, 100])


# Training the model

In [33]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),
                            lr=0.001,
                            momentum=0.9,
                            weight_decay=5e-4)

In [38]:
from tqdm.auto import tqdm
from timeit import default_timer as timer
epochs = 10

start_timer = timer()
for epoch in tqdm(range(epochs)):
  print(f"Epoch: {epoch}\n--------------------------------------------")
  train_step(model=model,
             train_dataloader=train_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             accuracy_fn=accuracy_fn,
             device=device)
  test_step(model=model,
            test_dataloader=test_dataloader,
            loss_fn=loss_fn,
            accuracy_fn=accuracy_fn,
            device=device)
end_timer = timer()

model_train_time = train_time(start_timer, end_timer, device)


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 0
--------------------------------------------


Training in progress:   0%|          | 0/1562 [00:00<?, ?it/s]

Train loss: 1.2101 | Train acc: 65.15%


  0%|          | 0/313 [00:00<?, ?it/s]

...

Test loss: 1.2060 | Test acc: 65.72%
Epoch: 9
--------------------------------------------


Training in progress:   0%|          | 0/1562 [00:00<?, ?it/s]

Train loss: 1.1424 | Train acc: 66.98%


  0%|          | 0/313 [00:00<?, ?it/s]

Test loss: 1.1863 | Test acc: 65.94%
Took 497.4553107689999 seconds on device cuda


In [39]:
model_results = eval_model(model=model,
           data_loader=test_dataloader,
           loss_fn=loss_fn,
           accuracy_fn=accuracy_fn,
           device=device)
model_results

  0%|          | 0/313 [00:00<?, ?it/s]

{'model_name': 'TheImageClassification_inatorV0',
 'model_acc': 65.94448881789137,
 'model_loss': 1.1862568482042501}

In [35]:
torch.save(model.state_dict(), f="model.pth")